In [325]:
import pyttsx3
import speech_recognition as sr
import webbrowser
import datetime
import pywhatkit
import os
import yfinance as yf
import pyjokes
from twilio.rest import Client
from ecapture import ecapture as ec
import wikipedia
import tkinter
import json
import urllib.request
from ecapture import ecapture as ec
import wolframalpha # to calculate strings into formula
import selenium.webdriver as webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException
import signal

In [326]:
#Listen to our microphone and return the audio as text using google

def transform():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.pause_threshold = 1.2
        said = r.listen(source)
        try:
            print('I am listening...')
            q = r.recognize_google(said, language="en")
            return q
        except sr.UnknownValueError:
            print('Sorry I did not understand')
            return "I am waiting..."
        except sr.RequestError:
            print('Sorry the service is down')
            return 'I am waiting...'
        except:
            return 'I am waiting..'
    return q

In [327]:
def speaking(message):
    engine =pyttsx3.init()
    engine.say(message)
    engine.runAndWait()

In [328]:
engine =pyttsx3.init()
for voice in engine.getProperty('voices'):
    print(voice)

<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0
          name=Microsoft David Desktop - English (United States)
          languages=[]
          gender=None
          age=None>
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-GB_HAZEL_11.0
          name=Microsoft Hazel Desktop - English (Great Britain)
          languages=[]
          gender=None
          age=None>
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
          name=Microsoft Zira Desktop - English (United States)
          languages=[]
          gender=None
          age=None>


In [329]:
#Change voice of bot by using id
id = 'HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0'
engine.setProperty('voice',id)
engine.say('hello World')
engine.runAndWait()

In [330]:
#return the weekday name
def query_day():
    day = datetime.date.today()
    #print(day)
    weekday = day.weekday()
    #print(weekday)
    mapping = {
        0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thurday',4:'Friday',5:'Saturday',6:'Sunday'
    }
    
    try:
        speaking(f'Today is {mapping[weekday]}')
    except:
        pass

In [309]:
#return the time
def query_time():
    time = datetime.datetime.now().strftime('%I:%M:%S')
    #print(time)
    speaking(f" The time is {time[1]} o'clock and {time[3:5]} minutes ")
    

In [310]:
#Intro greeting at statrtup
def whatsup():
    speaking('''Hi, I am Julia 1 point o.
    How may I help you''')

In [311]:
def set_reminder():
    speaking("What should I remind you about?")
    reminder = transform()
    speaking("When do you want to be reminded? Please say the time in hours and minutes.")
    reminder_time = transform()
    try:
        hour, minute = map(int, reminder_time.split())
        now = datetime.datetime.now()
        reminder_datetime = now.replace(hour=hour, minute=minute)
        if now > reminder_datetime:
            reminder_datetime += datetime.timedelta(days=1)
        speaking(f"Alright, I will remind you about '{reminder}' at {hour:02d}:{minute:02d}.")
        while True:
            if datetime.datetime.now() >= reminder_datetime:
                speak(engine, f"Reminder: {reminder}")
                break
    except ValueError:
        speaking("Sorry, I couldn't understand the time you provided. Please try again.")


In [312]:
def create_todo_list():
    todo_list = []
    speaking("Let's create a to-do list. Please say the tasks one by one. Say 'done' when you're finished.")
    while True:
        task = transform().lower()
        if task == "done":
            break
        todo_list.append(task)
        speaking(f"Added: {task}")
    speaking("Here's your to-do list:")
    for task in todo_list:
        speaking(task)

In [331]:
def search_web():
    user_agent = ' Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.48'
    edge_driver_path = "C:\\Users\\devin\\Downloads\\edgedriver_win64\\msedgedriver.exe"
    edge_service = Service(edge_driver_path)
    edge_options = Options()
    edge_options.use_chromium = True
    edge_options.add_experimental_option("detach", True)
    edge_options.add_argument(f'user-agent={user_agent}')
    whatsup()
    start = True
    while(start):
        q = transform().lower()
        if q:
            driver = webdriver.Edge(service=edge_service, options=edge_options)
            driver.implicitly_wait(1)
            driver.maximize_window()
            try:
                if 'youtube' in q:
                    try:
                        speaking("Opening in youtube")
                        indx =q.split().index('youtube')
                        driver.get('https://www.youtube.com/')
                        search_bar = driver.find_element(By.NAME,'search_query')
                        search_bar.send_keys(q)
                        search_bar.submit()
                        # submit search form
                        search_form = driver.find_element(By.ID, 'search-form')
                        search_form.submit()
                        video_link = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="video-title"]')))
                        video_link.click()
                        continue
                    except (NoSuchElementException, NoSuchWindowException) as e:
                        driver.quit()

                elif 'wikipedia' in q:
                    speaking('Checking wikipedia')
                    q = q.replace("wikipedia","")
                    result = wikipedia.summary(q,sentences=2)
                    speaking('Found on wikipedia')
                    try:
                        # Search for the page with title "India"
                        page = wikipedia.page(q)
                        driver.get(page.url)
                        speaking(result)

                    except wikipedia.exceptions.PageError:
                        print("Page not found.")
                    except wikipedia.exceptions.WikipediaException as e:
                        print("An error occurred: ", e)
                    
                    continue

                elif 'google' in q:
                    try:
                        speaking("Opening in Google")
                        indx = q.split().index('google')
                        query = ' '.join(q.split()[indx+1:])
                        driver.get('https://www.google.com/')
                        search_bar = driver.find_element(By.NAME,'q')
                        search_bar.send_keys(query)
                        search_bar.submit()
                        # submit search form
                        #search_form = driver.find_element(By.ID, 'search-form')
                        #search_form.submit()
                        continue
                    except (NoSuchElementException, NoSuchWindowException) as e:
                        continue
                    
                elif 'shutdown ' in q or 'exit' in q:
                    speaking('Ok I am shutting down')
                    break
                elif 'search web' in q:
                    pywhatkit.search(q)
                    speaking('that is what I found')
                    continue
                
                elif 'day' in q:
                    query_day()
                    continue

                elif 'time' in q:
                    query_time()
                    continue
                elif 'play music' in q:
                    speaking(f'playing {q}')
                    pywhatkit.playonyt(q)
                    continue
                elif 'your name' in q:
                    speaking('I am Julia. Your personal assistant')
                    continue
                elif "remainder" in q:
                    set_reminder()
                    continue
                    
                elif "to-do" in q or "todo" in q:
                    create_todo_list()
                    continue
                    
                elif 'start webbrowser' in q:
                    speaking('opening browser')
                    webbrowser.open('https://www.google.com')
                    continue
            
                elif 'joke' in q:
                    speaking(pyjokes.get_joke())
                    continue

                elif 'stock price' in q:
                    search = q.split("of")[-1].strip()
                    lookup = {'apple':'AAPL', 'amazon':'AMZN',
                             'google':'GOOGL'}
                    try:
                        stock = lookup[search]
                        stock = yf.Ticker(stock)
                        currentprice = stock.info('regularMarketPrice')
                        speaking(f'found it,the price for {search} is {currentprice}')
                    except:
                        speaking(f'sorry I have no data for {search}')
                        continue

                elif 'news' in q:
                    try:
                        jsonObj =urllib.request.urlopen('https://newsapi.org/v2/top-headlines?country=in&apiKey=e2681c3ebb754ff98e662c736a2f65c7')
                        data = json.load(jsonObj)
                        i = 1

                        speaking('here are some top news from the times of india')
                        print('''=============== TIMES OF INDIA ============'''+ '\n')

                        for item in data['articles']:

                            print(str(i) + '. ' + item['title'] + '\n')
                            print(item['description'] + '\n')
                            speaking(str(i) + '. ' + item['title'] + '\n')
                            i += 1
                    except Exception as e:
                        print(str(e))
                        continue

                elif "text file" in q:
                    speaking("Showing Notes")
                    file = open("Voice_Assistant_Bot.ipynb", "r")
                    print(file.read())
                    speaking(file.read(10))

                elif "where is" in q:
                    query = q.replace("where is", "")
                    location = query
                    url = f"https://www.google.com/maps/search/{location}"
                    speaking("User asked to Locate")
                    speaking(location)
                    webbrowser.open(url)
                    continue

                    
                else:
                    speaking('I am waiting for your command')
                    continue

            except (NoSuchElementException, NoSuchWindowException) as e:
                driver.quit()


In [ ]:
search_web()

I am listening...
